<a href="https://colab.research.google.com/github/maratborodin/spark_streaming/blob/main/hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark[sql] findspark --user

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 42 kB/s 
     |████████████████████████████████| 199 kB 59.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=8e270199ad44bb0231d9e0b0ff3d926d06c6703f164ef1c926040a08a6515282
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [1]:
# !pip install pyspark[sql] findspark --user

import sys

import findspark
findspark.init()
findspark.find()

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *


spark = SparkSession.builder \
    .appName("Spark Streaming") \
    .config("spark.driver.extraJavaOptions", "-Dcom.sun.net.ssl.checkRevocation=false") \
    .config("spark.executor.extraJavaOptions", "-Dcom.sun.net.ssl.checkRevocation=false") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:3.2.1") \
    .config("spark.sql.streaming.forceDeleteTempCheckpointLocation", "true") \
    .config("spark.executor.instances", "1") \
    .config("spark.executor.memory", "512mb") \
    .getOrCreate()

In [2]:
spark

In [3]:
df = spark.range(10)
sdf = spark \
    .readStream \
    .format("rate") \
    .load() #.option("param", "value")\.option("param", "value") \.option("param", "value") \

print(f'Is streaming: df - \'{df.isStreaming}\', sdf - \'{sdf.isStreaming}\'')

Is streaming: df - 'False', sdf - 'True'


In [4]:
sdf = spark \
    .readStream \
    .format("rate") \
    .load()
sdf

DataFrame[timestamp: timestamp, value: bigint]

In [5]:
sdf.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- value: long (nullable = true)



In [6]:
sdf.explain(True)

== Parsed Logical Plan ==
StreamingRelationV2 org.apache.spark.sql.execution.streaming.sources.RateStreamProvider@5d59488, rate, org.apache.spark.sql.execution.streaming.sources.RateStreamTable@298f34dd, [], [timestamp#6, value#7L]

== Analyzed Logical Plan ==
timestamp: timestamp, value: bigint
StreamingRelationV2 org.apache.spark.sql.execution.streaming.sources.RateStreamProvider@5d59488, rate, org.apache.spark.sql.execution.streaming.sources.RateStreamTable@298f34dd, [], [timestamp#6, value#7L]

== Optimized Logical Plan ==
StreamingRelationV2 org.apache.spark.sql.execution.streaming.sources.RateStreamProvider@5d59488, rate, org.apache.spark.sql.execution.streaming.sources.RateStreamTable@298f34dd, [], [timestamp#6, value#7L]

== Physical Plan ==
StreamingRelation rate, [timestamp#6, value#7L]



In [7]:
def create_console_sink(sdf):
    return sdf \
            .writeStream \
            .format("console") \
            .trigger(processingTime="10 seconds") \
            .option("truncate", "false") \
            .option("numRows", "10")

In [8]:
sink = create_console_sink(sdf)
sink

In [10]:
sq = sink.start()
sq.isActive

True

In [11]:
sq.stop()